In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F

import numpy as np

import torchvision
from torchvision import datasets, models, transforms


import time
import copy
import math
import matplotlib.pyplot as plt
import os

'1.9.0'

In [3]:
torch.manual_seed(123)

In [177]:
args = {
    
}

In [178]:
if torch.cuda.is_available():
    args['device'] = torch.device('cuda')
else:
    args['device'] = torch.device('cpu')
    
print(args['device'])


cpu


In [179]:
#base_dir = "..\Keras\ProjetoCatsDogs\cats_and_dogs"
base_dir = "..\Keras\cats_and_dogs_filtered"
train_dir = os.path.join(base_dir, "train")
validation_dir = os.path.join(base_dir, "validation")

tr_dir_cats = os.path.join(train_dir, "cats")
tr_dir_dogs = os.path.join(train_dir, "dogs")
val_dir_cats = os.path.join(validation_dir, "cats")
val_dir_dogs = os.path.join(validation_dir, "dogs")

num_tr_cats = len(os.listdir(tr_dir_cats))
num_tr_dogs = len(os.listdir(tr_dir_dogs))
num_val_cats = len(os.listdir(val_dir_cats))
num_val_dogs = len(os.listdir(val_dir_dogs))

num_tr = num_tr_cats + num_tr_dogs
num_val = num_val_cats + num_val_dogs

In [180]:
num_tr_cats, num_tr_dogs, num_val_cats, num_val_dogs

(1000, 1000, 500, 500)

In [181]:
transform = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])


In [182]:
dataset_train = datasets.ImageFolder(train_dir, transform=transform)

In [183]:
dataset_train.classes, len(dataset_train.imgs)

(['cats', 'dogs'], 2000)

In [184]:
dataset_val = datasets.ImageFolder(validation_dir, transform=transform)

In [185]:
dataset_val.classes, len(dataset_val.imgs)

(['cats', 'dogs'], 1000)

In [186]:
batch_size = 64

In [187]:
train_set = DataLoader(dataset=dataset_train, shuffle=True, batch_size=batch_size)

In [188]:
val_set = DataLoader(dataset=dataset_val, shuffle=False, batch_size=batch_size)

In [189]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        self.pool = nn.MaxPool2d(2,2)
        self.dropout = nn.Dropout(p=0.2)
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=4)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=4)
        self.conv3 = nn.Conv2d(in_channels=12, out_channels=14, kernel_size=4)
        self.conv4 = nn.Conv2d(in_channels=14, out_channels=16, kernel_size=4)
        self.conv5 = nn.Conv2d(in_channels=16, out_channels=20, kernel_size=4)
        
        self.fc1 = nn.Linear(in_features=20*4*4, out_features=250)
        self.fc2= nn.Linear(in_features=250, out_features=200)
        self.fc3= nn.Linear(in_features=200, out_features=50)
        self.fc4= nn.Linear(in_features=50, out_features=10)
        self.fc5= nn.Linear(in_features=10, out_features=2)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = self.pool(F.relu(self.conv5(x)))
        
        x = x.reshape(-1, 20 * 4 * 4)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        x = self.dropout(F.relu(self.fc4(x)))
        x = self.fc5(x)
        return x
    
net = Model().to(args['device'])

print(net)

Model(
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.2, inplace=False)
  (conv1): Conv2d(3, 6, kernel_size=(4, 4), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(4, 4), stride=(1, 1))
  (conv3): Conv2d(12, 14, kernel_size=(4, 4), stride=(1, 1))
  (conv4): Conv2d(14, 16, kernel_size=(4, 4), stride=(1, 1))
  (conv5): Conv2d(16, 20, kernel_size=(4, 4), stride=(1, 1))
  (fc1): Linear(in_features=320, out_features=250, bias=True)
  (fc2): Linear(in_features=250, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=50, bias=True)
  (fc4): Linear(in_features=50, out_features=10, bias=True)
  (fc5): Linear(in_features=10, out_features=2, bias=True)
)


In [190]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay = 1e-5)

In [194]:
net.train()

for epoch in range(15):
    total_correct = 0.0
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_set):
        inputs, labels = inputs.to(args['device']), labels.to(args['device'])
        output = net(inputs)
        output_idx = torch.argmax(output, dim=1)
        total_correct += (labels == output_idx).sum().item()
        optimizer.zero_grad()
        loss = criterion(output, labels)
        running_loss += loss.item() * inputs.size(0)
        loss.backward()
        optimizer.step()
        
    print(f'Epoch: {epoch} Loss: {running_loss/num_tr} Accuracy:{(total_correct/num_tr)*100}%')
    
print('Finished training')

Epoch: 0 Loss: 0.6902781391143799 Accuracy:52.15%
Epoch: 1 Loss: 0.6836738672256469 Accuracy:57.15%
Epoch: 2 Loss: 0.6662602910995483 Accuracy:63.5%
Epoch: 3 Loss: 0.6906118755340576 Accuracy:54.05%
Epoch: 4 Loss: 0.6667960734367371 Accuracy:62.35000000000001%
Epoch: 5 Loss: 0.6404072465896606 Accuracy:65.2%
Epoch: 6 Loss: 0.6089389753341675 Accuracy:67.60000000000001%
Epoch: 7 Loss: 0.5883030045032501 Accuracy:68.95%
Epoch: 8 Loss: 0.5752456364631653 Accuracy:69.95%
Epoch: 9 Loss: 0.5470855102539063 Accuracy:74.0%
Epoch: 10 Loss: 0.539686381816864 Accuracy:72.39999999999999%
Epoch: 11 Loss: 0.5079318909645081 Accuracy:75.44999999999999%
Epoch: 12 Loss: 0.48352094984054567 Accuracy:78.60000000000001%
Epoch: 13 Loss: 0.4540327730178833 Accuracy:79.10000000000001%
Epoch: 14 Loss: 0.44797103881835937 Accuracy:79.4%
Finished training
